# Demo for The Neuro-symbolic Model

This notebook offers an interactive application for testing out the concept of a neuro-symbolic classifier. The user can choose a class from the avilable dataset, and a random test image will be passed to the model. The custom model will classify the image and display the decision rule used in the making of the decision.

## Imports

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import gradio as gr
import torch
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F

In [3]:
!wget "https://github.com/MatTheTab/neuro-symbolic-image-classifier/raw/refs/heads/main/models/neuro_symbolic_classifier.pkl"

--2025-02-18 18:34:36--  https://github.com/MatTheTab/neuro-symbolic-image-classifier/raw/refs/heads/main/models/neuro_symbolic_classifier.pkl
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MatTheTab/neuro-symbolic-image-classifier/refs/heads/main/models/neuro_symbolic_classifier.pkl [following]
--2025-02-18 18:34:36--  https://raw.githubusercontent.com/MatTheTab/neuro-symbolic-image-classifier/refs/heads/main/models/neuro_symbolic_classifier.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2119908 (2.0M) [application/octet-stream]
Saving to: ‘neuro_symbolic_classifier.pkl’

neuro_symbolic_c

## Redefnitions (Necessary for Imports)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
class MultiLabelCNN(nn.Module):
    """
    A convolutional neural network for multi-label classification.

    Attributes:
    conv1, conv2, conv3 (nn.Conv2d): Convolutional layers.
    bn1, bn2, bn3 (nn.BatchNorm2d): Batch normalization layers.
    pool (nn.MaxPool2d): Max pooling layer.
    fc1, fc2, fc3 (nn.Linear): Fully connected layers.
    """
    def __init__(self, num_classes=10):
        """
        Initializes the MultiLabelCNN model.

        Parameters:
        num_classes (int, optional): Number of output classes. Defaults to 10.
        """
        super(MultiLabelCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        expected_size = self.get_expected_size()
        self.fc1 = nn.Linear(expected_size, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, num_classes)

    def get_expected_size(self):
        """
        Computes the output size after convolution and pooling layers.

        Returns:
        int: Flattened feature size before passing into fully connected layers.
        """
        device = next(self.parameters()).device
        random_input = torch.rand((1, 3, 32, 32), device=device)

        x = self.pool(F.relu(self.bn1(self.conv1(random_input))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        return x.view(x.size(0), -1).size(1)

    def forward(self, x):
        """
        Defines the forward pass of the CNN.

        Parameters:
        x (Tensor): Input tensor of shape (batch_size, 3, height, width).

        Returns:
        Tensor: Output logits for each class.
        """
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [6]:
class NeuroSymbolicClassifier:
    """
    A hybrid classifier that combines neural network predictions with symbolic rule-based reasoning.

    The model first predicts a set of feature probabilities using the neural network, then converts them into
    binary values using a threshold. Based on this feature vector, a symbolic decision tree is used to predict
    the class. If a matching rule is found, it is returned; otherwise, a default message is returned.

    Parameters:
    neural_model (nn.Module): A trained neural network model for predicting feature probabilities.
    rules (list): A list of symbolic rules to be applied based on the predicted class.
    tree (sklearn.tree.DecisionTreeClassifier): A decision tree model used for classification based on feature vector.
    threshold (float): Threshold value for converting feature probabilities to binary values (default is 0.5).
    device (str): Device to run the model on, either "cpu" or "cuda" (default is "cpu").
    """
    def __init__(self, neural_model, rules, tree, threshold=0.5, device="cpu"):
        """
        Initializes the NeuroSymbolicClassifier.

        Parameters:
        neural_model (nn.Module): The trained neural network.
        rules (list): The set of rules to use with symbolic reasoning.
        tree (sklearn.tree.DecisionTreeClassifier): The decision tree for class prediction based on the binary feature vector.
        threshold (float): Threshold to determine the binary classification of each feature.
        device (str): The device on which the neural model is run (either "cpu" or "cuda").
        """
        neural_model.to(device)
        self.neural_model = neural_model
        self.rules = rules
        self.tree = tree
        self.threshold = threshold

    def _convert_to_binary(self, feature_probs):
        """
        Converts predicted feature probabilities to binary values based on a threshold.

        Parameters:
        feature_probs (list): The list of predicted feature probabilities from the neural network.

        Returns:
        tuple: A tuple of binary values (0 or 1) based on the threshold.
        """
        return tuple(int(val >= self.threshold) for val in feature_probs)

    def _find_matching_rule(self, predicted_class):
        """
        Searches for a matching symbolic rule corresponding to the predicted class.

        Parameters:
        predicted_class (str): The predicted class from the decision tree.

        Returns:
        str: The matching rule, or "NO MATCHING RULE" if no rule is found.
        """
        for rule in self.rules:
            if predicted_class in rule:
                return rule
        return "NO MATCHING RULE"

    def predict(self, image):
        """
        Makes a prediction using the neural model, decision tree, and symbolic rules.

        Parameters:
        image (ndarray or tensor): The input image for which a prediction is made.

        Returns:
        tuple: A tuple containing the predicted class and the applied rule (if any).
        """
        image = torch.tensor(image)
        image.to(device)
        self.neural_model.eval()
        with torch.no_grad():

            feature_probs = self.neural_model(image).squeeze().tolist()
        feature_vector = np.array(self._convert_to_binary(feature_probs), dtype=np.int8)
        predicted_class = str(self.tree.predict([feature_vector])[0])
        rule = self._find_matching_rule(predicted_class)
        return predicted_class, rule

## The Application

In [7]:
with open("neuro_symbolic_classifier.pkl", "rb") as f:
    hybrid_classifier = pickle.load(f)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

testset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
classes = testset.classes
imgs_per_class = {}
for img_class in classes:
    imgs_per_class[img_class] = []

for img, img_class in testset:
    imgs_per_class[classes[img_class]].append(img.numpy().copy())
del testset

def classify_image(selected_class):
    '''
    Function selects a random image from the specified class, processes it,
    classifies it using a hybrid classifier, and displays the image with
    the predicted class and applied rule.

    Parameters:
    selected_class (str or int): The class label for which an image is selected.

    Returns:
    matplotlib.figure.Figure: A figure displaying the classified image with title annotations.
    '''
    class_images = imgs_per_class[selected_class]
    image = random.choice(class_images)
    image_input = np.expand_dims(image, axis=0)
    predicted_class, applied_rule = hybrid_classifier.predict(image_input)
    img_display = image.transpose((1, 2, 0))
    img_display = img_display * 0.5 + 0.5
    fig, ax = plt.subplots()
    ax.imshow(img_display)
    ax.axis("off")
    ax.set_title(f"Predicted: {predicted_class}\nRule: {applied_rule}", fontsize=7)
    return fig

app_description = """
This application demonstrates a **hybrid neuro-symbolic classifier** trained on the CIFAR-10 dataset.
Users can select a class from the dropdown menu, and the app will randomly pick an image from the test set belonging to this class.
The image is then fed to the model using both a neural network and a set of symbolic rules to correctly classify and describe the image.
This solutions combines the strengths of neural networks and symbolic reasoning, enjoy!
"""

interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Dropdown(choices=classes, label="Select a class"),
    outputs=gr.Plot(label="Model Prediction"),
    title="Neuro-Symbolic Image Classifier",
    description=app_description
)

interface.launch()


100%|██████████| 170M/170M [00:08<00:00, 19.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1867650ced08122042.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
